# Musicians - Easy

## Tables used in the musicians database

band(**band_no**, band_name, band_home, band_type, b_date, band_contact)

composer(**comp_no**, comp_is, comp_type)

composition(**c_no**, comp_date, c_title, c_in)

concert(**concert_no**, concert_venue, concert_in, con_date, concert_orgniser)

has_composed(**_cmpr_no_**, **_cmpn_no_**)

musician(**m_no**, m_name, born, died, born_in, living_in)

performance(**_pfrmnc_no_**, gave, performed, conducted_by, performed_in)

performer(**perf_no**, perf_is, instrument, perf_type)

place(**place_no**, place_town, place_country)

plays_in(**_player_**, **band_id**)

- **musician**

m_no | m_name | born | died | born_in | living_in
-:|-----------|------|------|--------:|-----:
1 | Fred Bloggs | 02/01/48 |  | 1 | 2
2 | John Smith | 03/03/50 |  | 3 | 4
3 | Helen Smyth | 08/08/48 |  | 4 | 5
4 | Harriet Smithson | 09/05/1909 | 20/09/80 | 5 | 6
5 | James First | 10/06/65 |  | 7 | 7
... | | | | | 

- **place**

place_no | place_town | place_country
--------:|------------|---------
1 | Manchester | England
2 | Edinburgh | Scotland
3 | Salzburg | Austria
4 | New York | USA
5 | Birmingham | England
... | | 

- **performer**

perf_no | perf_is | instrument | perf_type
-:|--:|--------|---------------------
1 | 2 | violin | classical
2 | 4 | viola | classical
3 | 6 | banjo | jazz
4 | 8 | violin | classical
5 | 12 | guitar | jazz
...| | |

- **composer**

comp_no | comp_is | comp_type
-:|--:|----------------
1 | 1 | jazz
2 | 3 | classical
3 | 5 | jazz
4 | 7 | classical
5 | 9 | jazz
... | |

- **band**

band_no | band_name | band_home | band_type | b_date | band_contact
-:|-----|--:|-----------|----------|----:
1 | ROP | 5 | classical | 30/01/01 | 11
2 | AASO | 6 | classical |  | 10
3 | The J Bs | 8 | jazz |  | 12
4 | BBSO | 9 | classical |  | 21
5 | The left Overs | 2 | jazz |  | 8
... | | | | |

- **plays_in**

player | band_id
-:|---:
1 | 1
1 | 7
3 | 1
4 | 1
4 | 7
... |

- **composition**

c_no | comp_date | c_title | c_in
-:|----------|--------|-------:
1 | 17/06/75 | Opus 1 | 1
2 | 21/07/76 | Here Goes | 2
3 | 14/12/81 | Valiant Knight | 3
4 | 12/01/82 | Little Piece | 4
5 | 13/03/85 | Simple Song | 5
... | | |

- **has_composed**

cmpr_no | cmpn_no
-:|---:
1 | 1
1 | 8
2 | 11
3 | 2
3 | 13
... |

- **concert**

concert_no | concert_venue | concert_in | con_date | concert_orgniser
-:|------------------|--:|----------|----:
1 | Bridgewater Hall | 1 | 06/01/95 | 21
2 | Bridgewater Hall | 1 | 08/05/96 | 3
3 | Usher Hall | 2 | 03/06/95 | 3
4 | Assembly Rooms | 2 | 20/09/97 | 21
5 | Festspiel Haus | 3 | 21/02/95 | 8
... | | | |

- **performance**

pfrmnc_no | gave | performed | conducted_by | performed_in
-:|--:|--:|---:|-----------------------:
1 | 1 | 1 | 21 | 1
2 | 1 | 3 | 21 | 1
3 | 1 | 5 | 21 | 1
4 | 1 | 2 | 1 | 2
5 | 2 | 4 | 21 | 2
... | | | |

Here is the data model of the musician tables

![](../src/img/musician_str.svg)

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


 ········


In [2]:
composer = pd.read_sql_table('composer', engine)
composition = pd.read_sql_table('composition', engine)
concert = pd.read_sql_table('concert', engine)
has_composed = pd.read_sql_table('has_composed', engine)
musician = pd.read_sql_table('musician', engine)
performance = pd.read_sql_table('performance', engine)
performer = pd.read_sql_table('performer', engine)
place = pd.read_sql_table('place', engine)
plays_in = pd.read_sql_table('plays_in', engine)


## 1.
**Give the organiser's name of the concert in the Assembly Rooms after the first of Feb, 1997.**

In [3]:
(concert.loc[(concert['concert_venue']=='Assembly Rooms') & 
             (concert['con_date']>'1997-02-01')]
 .merge(musician, left_on='concert_orgniser', right_on='m_no')
 [['m_name']])

,m_name
0,James Steeple


## 2.
**Find all the performers who played guitar or violin and were born in England.**

In [4]:
(performer.loc[performer['instrument'].isin(['violin', 'guitar'])]
 .merge(musician, left_on='perf_is', right_on='m_no')
 .merge(place.loc[place['place_country']=='England'],
        left_on='born_in', right_on='place_no')
 [['m_name']].sort_values('m_name'))

,m_name
3,Alan Fluff
1,Davis Heavan
2,Harriet Smithson
0,Harry Forte
5,James First
4,Theo Mengel


## 3.
**List the names of musicians who have conducted concerts in USA together with the towns and dates of these concerts.**

In [5]:
(concert.merge(place.loc[place['place_country']=='USA'],
               left_on='concert_in', right_on='place_no')
 .merge(performance, left_on='concert_no', right_on='performed_in')
 .merge(musician, left_on='conducted_by', right_on='m_no')
 [['m_name', 'place_town', 'con_date']].drop_duplicates()
 .sort_values('m_name'))

,m_name,place_town,con_date
0,James Steeple,New York,1997-06-15


## 4.
**How many concerts have featured at least one composition by Andy Jones? List concert date, venue and the composition's title.**

In [6]:
(concert.merge(composition, left_on='concert_no', right_on='c_in')
 .merge(has_composed, left_on='c_no', right_on='cmpn_no')
 .merge(composer, left_on='cmpr_no', right_on='comp_no')
 .merge(musician.loc[musician['m_name']=='Andy Jones'], 
        left_on='comp_is', right_on='m_no')
 [['con_date', 'concert_venue', 'c_title']])

,con_date,concert_venue,c_title
0,1996-05-08,Bridgewater Hall,A Simple Piece


## 5.
**List the different instruments played by the musicians and avg number of musicians who play the instrument.**

In [7]:
a = (performer.merge(musician, left_on='perf_is', right_on='m_no')
     .pivot_table(columns='m_name', index='instrument', values='m_no', 
                  aggfunc='count', fill_value=0, margins=True))
a

m_name,Alan Fluff,Davis Heavan,Elsie James,Harriet Smithson,Harry Forte,Helen Smyth,James First,James Quick,Jeff Dawn,John Smith,Louise Simpson,Sue Little,Theo Mengel,All
instrument,,,,,,,,,,,,,,
banjo,0,0,0,0,0,0,0,0,0,0,0,0,1,1
bass,0,0,0,0,0,0,0,0,1,1,1,0,0,3
cello,0,1,1,0,0,0,0,0,0,0,0,1,0,3
clarinet,0,0,0,0,1,0,0,0,0,0,0,0,0,1
cornet,0,0,0,0,0,0,0,0,0,0,1,0,0,1
drums,0,0,0,0,1,0,0,0,0,0,0,0,1,2
flute,0,0,0,0,0,0,0,1,0,1,0,0,0,2
guitar,0,1,1,0,0,0,0,0,0,0,0,0,0,2
horn,0,0,0,0,0,1,0,0,0,0,0,0,0,1
